# Versuch 2 - Kalibrierung von Digitalkameras

Dominik Bartsch, Stefan Ptacek, Niklas Kaiser

In [1]:
!python -m pip install opencv-python
import cv2
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


## 1. Aufnahme und Analyse eines Grauwertkeiles
### 1.1 Einlesen der Werte aus der Kamera

In [4]:
# -------- Aufgabe1.1 -------- #

cap = cv2.VideoCapture(0)

for i in range(10):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("./data/fix" + str(i) + ".png", gray)

print("frame width: " + str(cap.get(3)))
print("frame height: " + str(cap.get(4)))
print("--------------------------------")
print("brightness: " + str(cap.get(10)))
print("contrast: " + str(cap.get(11)))
print("saturation: " + str(cap.get(12)))
print("--------------------------------")
print("gain: " + str(cap.get(14)))
print("exposure: " + str(cap.get(15)))
print("--------------------------------")
print("white balance: " + str(cap.get(17)))

cap.release()
#cv2.destroyAllWindows()

[ WARN:0@11.575] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@11.575] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.8.1) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


### 1.2 Mittelwert und die Standartabweichung

In [2]:
image = cv2.imread("./data/dark1.png")
image = image.astype('float32')

schwarz = image[:,0:50]
dunkelgrau = image[:,150:200]
mittelgrau = image[:,300:350]
hellgrau = image[:,500:550]
weiß = image[:,550:600]

cv2.imwrite("./data/changed/black.png", schwarz.astype('uint8'))
cv2.imwrite("./data/changed/dark1.png", dunkelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark2.png", mittelgrau.astype('uint8'))
cv2.imwrite("./data/changed/dark3.png", hellgrau.astype('uint8'))
cv2.imwrite("./data/changed/white.png", weiß.astype('uint8'))

print("\t\t| std \t\t| mean")
print("-----------------------------------------------------")
print("schwarz \t| %f \t| %f" %(np.std(schwarz),np.mean(schwarz)))
print("dunkelgrau \t| %f \t| %f" %(np.std(dunkelgrau),np.mean(dunkelgrau)))
print("mittelgrau \t| %f \t| %f" %(np.std(mittelgrau),np.mean(mittelgrau)))
print("hellgrau \t| %f \t| %f" %(np.std(hellgrau),np.mean(hellgrau)))
print("weiß \t\t| %f \t| %f" %(np.std(weiß),np.mean(weiß)))

		| std 		| mean
-----------------------------------------------------
schwarz 	| 2.969366 	| 29.091000
dunkelgrau 	| 2.326184 	| 67.090790
mittelgrau 	| 1.746090 	| 113.020500
hellgrau 	| 2.471456 	| 150.536377
weiß 		| 4.311863 	| 195.475830


### Mittelwert und die Standardabweichung

In [6]:
img_arr = []


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread("./data/" + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


for i in range(10):
    img_read = cv2.imread("./data/dark" + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
    img_arr.append(img_read)

img = np.mean(img_arr, axis=0, dtype=float)
print(img)
sub_arrays = []
slicing_index = []

step = 20
threshold = 20
for i in range(0, len(img[0]) - step, step):
    if abs(int(img[0, i + step]) - int(img[0, i])) > threshold:
        # print("Slicing at index: " + str(i) + " with value: "
        #       + str(img[0][i]) + " and next value: " + str(img[0][i + 1]))
        slicing_index.append(i)
        sub_arrays.append(img[:, i-99:i])

slicing_index.append(len(img[0]))  # add last index
sub_arrays.append(img[:, len(img[0]) - 98:len(img[0]) - 1])

print(slicing_index)
print(sub_arrays)

res_mean = []
res_std = []
i = 0
for x in sub_arrays:
    fixed_x = fix_noise(sub_arrays[0][:,90], sub_arrays[4][:,90], x[:, 90])
    mean = np.round(np.mean(x), 2)
    mean_fixed = np.round(np.mean(fixed_x), 2)
    res_mean.append(mean)
    std = np.round(np.std(x), 2)
    std_fixed = np.round(np.std(fixed_x), 2)
    res_std.append(std)

    
    print(f"Index: {i} , Mean: {mean}, Std: {std},  Fixed Mean: {mean_fixed}, Fixed Std: {std_fixed}")
    i += 1



# visualize sub arrays
#for i in range(len(sub_arrays)):
#    cv2.imshow("sub_arrays" + str(i), sub_arrays[i].astype(np.uint8))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

[[ 39.4  40.3  40.2 ... 190.6 189.7 189.7]
 [ 36.3  37.3  36.9 ... 191.7 191.7 191.7]
 [ 33.2  32.9  33.1 ... 192.9 193.1 193.1]
 ...
 [ 40.9  41.7  41.  ... 189.5 189.3 189.3]
 [ 56.   56.3  54.9 ... 189.3 188.9 188.9]
 [ 74.   73.4  71.4 ... 189.3 189.2 189.2]]
[120, 260, 400, 540, 640]
[array([[36.9, 37. , 37.4, ..., 34.8, 34.6, 36. ],
       [35.7, 35. , 34.7, ..., 34.9, 33.6, 34.1],
       [33.7, 32.7, 32. , ..., 33.8, 32.4, 32.5],
       ...,
       [38.2, 37.4, 36.8, ..., 34.7, 34.6, 34.3],
       [45.5, 44.7, 44. , ..., 37.2, 37. , 37.3],
       [62.3, 59.9, 58.5, ..., 41.9, 41.2, 42.2]]), array([[64.8, 64.8, 65.5, ..., 65.9, 66.6, 68.1],
       [64.9, 65.2, 64.9, ..., 65.2, 66.2, 67.3],
       [65.1, 66. , 65.8, ..., 65.3, 66.2, 66.3],
       ...,
       [65.8, 65.7, 65. , ..., 64.9, 66. , 66.1],
       [65.7, 65.2, 64.5, ..., 65.7, 65.8, 65. ],
       [66.1, 65.9, 66. , ..., 68.5, 67. , 66.2]]), array([[108. , 107.6, 108.3, ..., 111.7, 111.5, 111.2],
       [107.5, 107. , 106

## 2. Aufnahme eines Dunkelbildes

In [3]:
def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


def fix_noise(dark_img, light_img, img):
    img = img - dark_img
    light_img = (light_img - dark_img) / np.mean(light_img)
    img = img / light_img
    return img

In [ ]:
img_dark = get_image("./data/black")
img_light = get_image("./data/white")

res = fix_noise(img_dark, img_light, get_image("./data/dark"))
print(res)
cv2.imshow("res", res.astype(np.uint8))

alpha = 3  # Contrast control (1.0-3.0)
beta = 0  # Brightness control (0-100)

adjusted_dark = cv2.convertScaleAbs(img_dark, alpha=alpha, beta=beta)
adjusted_light = cv2.convertScaleAbs(img_light, alpha=alpha, beta=beta)

print("black")
cv2.imshow('original_black', img_dark)
cv2.imshow('adjusted_black', adjusted_dark)

print("white")
cv2.imshow('original_white', img_light)
cv2.imshow('adjusted_white', adjusted_light)

cv2.waitKey(0)
#cv2.destroyAllWindows()

[[ 41.59738094  42.37635731  42.29247895 ... 204.95645064 203.77901128
  203.77901128]
 [ 37.96022577  39.00596201  38.56837368 ... 205.18935457 205.39969732
  205.39969732]
 [ 34.35750909  34.0976898   34.30497059 ... 205.31737351 205.00833262
  205.00833262]
 ...
 [ 43.22469889  44.00343049  43.22112812 ... 200.16971733 201.07611039
  201.07611039]
 [ 59.15305631  59.46994768  57.9033446  ... 200.6682481  200.44752008
  200.44752008]
 [ 78.1665387   77.53275595  75.38205581 ... 200.56654093 201.17433498
  201.17433498]]
black
white


## 3. Aufnahme eines Weißbildes

In [9]:
imagedb = get_image("./data/black") #dunkelbild
# imagedb = imagedb.astype('float32')

imagewb = get_image("./data/white") #weißbild
# imagewb = imagewb.astype('float32')


eingangsbildname = "dark1"
imageub = cv2.imread("./data/" + eingangsbildname + ".png") #unbearbeitetes einlesebild
imageub = imageub.astype('float32')

darksubtract = np.subtract(imageub,imagedb) # Dunkelbild von Eingangsbild abziehen

cv2.imwrite("./data/changed/black_bearbeitet.png", darksubtract.astype('uint8'))

meanwhite = np.divide(imagewb, np.mean(imagewb)) # Weißbild normiert mit Mittelwert zu 1

cv2.imwrite("./data/changed/meanwhitebearbeitet.png", imagewb.astype('uint8'))

fin = darksubtract/meanwhite #np.divide(darksubtract, meanwhite) # Bearbeitetes Eingangsbild mit meanwhite dividieren

cv2.imwrite("./data/changed/" + eingangsbildname + "_bearbeitet.png", fin.astype('uint8'))

ValueError: operands could not be broadcast together with shapes (480,640,3) (480,640) 

### Kontrastmaximiertes Weißbild:
![](data/changed/meanwhitebearbeitet.png)

### Eingangsbild
![](data/dark1.png)

### Korrigiertes Eingangsbild
![](data/changed/dark1_bearbeitet.png)

## 4. Pixelfehler

In [9]:
def get_hot_pixels():
    dark_img = get_image("./data/black")
    return np.any(dark_img[:, :] > 0)


def get_stuck_pixels():
    dark_img = get_image("./data/black")
    light_img = get_image("./data/white")
    return np.any(dark_img[:, :] == light_img[:, :])


def get_dead_pixels():
    light_img = get_image("./data/white")
    return np.any(light_img[:, :] == 0)


def get_image(image_path):
    img_arr = []

    for i in range(10):
        img_read = cv2.imread(image_path + str(i) + ".png", cv2.IMREAD_GRAYSCALE)
        img_arr.append(img_read)

    return np.mean(img_arr, axis=0).astype(np.double)


print(get_hot_pixels())
print(get_stuck_pixels())
print(get_dead_pixels())

False
False
False
